In [1]:
!pip install tensorflow Cython matplotlib

In [2]:
!pip install simple-elmo

In [3]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-lih0w64t
  Running command git clone --filter=blob:none --quiet https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-lih0w64t
  Resolved https://github.com/lopuhin/python-adagram.git to commit cf3639f10d6a1efbcb602f45a1da89ef55ce5794
  Preparing metadata (setup.py) ... done


In [4]:
!pip install pymystem3 pymorphy2

In [5]:
!pip install corus

In [6]:
# библиотеки для работы с эмбеддингами
import adagram
from simple_elmo import ElmoModel

# обработка данных и ML
import pandas as pd
from lxml import html
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.cluster import *
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 1. Подготовка лексем

Я взяла 5 лексем из МАС: *ручка*, *коса*, *писать*, *свежий*, *больной*.

Немного модифицировала толкования (расшифровала сокращения, чтобы моделям было проще понять смысл).

In [7]:
definitions = {'ручка': [], 'коса': [], 'писать': [], 'свежий': [], 'больной': []}

In [8]:
definitions['ручка'].append('Уменьшительно-ласкательное к рука: Каждая из двух верхних конечностей человека от плечевого сустава до кончиков пальцев. Та же конечность от запястья до кончиков пальцев; кисть.')
definitions['ручка'].append('Часть предмета, за которую его держат или берут рукой. Приспособление у машин, аппаратов, служащее для приведения их в движение путем вращения, повертывания рукой.')
definitions['ручка'].append('Часть кресла, дивана, на которую, сидя, опираются локтем; подлокотник.')
definitions['ручка'].append('Письменная принадлежность — палочка, в которую вставляется перо, стержень.')

In [9]:
definitions['коса'].append('Заплетенные волосы.')
definitions['коса'].append('Ручное сельскохозяйственное орудие для скашивания травы и злаков в виде длинного изогнутого ножа, отточенного с одной стороны и прикрепленного к длинной рукоятке.')
definitions['коса'].append('Длинная узкая отмель, идущая от берега, или низменный узкий мыс.')
definitions['коса'].append('Узкая полоса чего-либо')

In [10]:
definitions['писать'].append('Изображать на бумаге или ином материале какие-либо знаки (буквы, цифры и т. п.). Уметь изображать буквы, слова и т. п. Быть годным для изображения каких-либо знаков (на бумаге или ином материале).')
definitions['писать'].append('Письменно составлять какой-либо текст. Владеть письменной речью какого-либо языка.')
definitions['писать'].append('Сообщать о чем-либо, высказывать что-либо письменно или печатно. Обращаться к кому-либо письменно, посылать письмо.')
definitions['писать'].append('Сочинять, создавать какое-либо словесное произведение. Изображать кого-, что-либо в литературном произведении. Заниматься литературной деятельностью, быть писателем.')
definitions['писать'].append('Сочинять, создавать музыкальное произведение.')
definitions['писать'].append('Создавать произведение живописи. Изображать кого-, что-либо в произведении живописи')

In [11]:
definitions['свежий'].append('Недавно или только что полученный, добытый, приготовленный и т. п. и не потерявший своих естественных хороших качеств. Только что собранный, срезанный (о плодах, растениях и т. п.). Не подвергшийся какой-либо обработке (солению, копчению, консервированию и т. п.), оставшийся в естественном виде.')
definitions['свежий'].append('Не бывший в употреблении, еще не применявшийся, не использованный.')
definitions['свежий'].append('Ничем не загрязненный, чистый, часто обновляющийся.')
definitions['свежий'].append('Восстановленный, обновленный (отдыхом, сном).')
definitions['свежий'].append('Не потерявший своей силы, яркости, блеска или естественной здоровой окраски. Звучный, звонкий, чистый (о голосе, смехе и т. п.). Отчетливо и ясно сохранившийся в памяти, не забытый.')
definitions['свежий'].append('Полный здоровья, сил (о человеке); цветущий. Свидетельствующий о здоровье.')
definitions['свежий'].append('Недавно или только что появившийся, возникший. Впервые или только что появившийся где-либо. Самый новый, последний, еще не известный кому-либо. Впервые высказанный, примененный; оригинальный.')
definitions['свежий'].append('Довольно холодный, прохладный.')
definitions['свежий'].append('Морское: Сильный (о ветре, волне). С сильным, холодным ветром (о погоде).')

In [12]:
definitions['больной'].append('Страдающий какой-либо болезнью; противоположное: здоровый.')
definitions['больной'].append('Свидетельствующий о болезни; болезненный.')
definitions['больной'].append('Неестественный, ненормальный.')

# 2. Подготовка корпуса

In [13]:
from corus import load_morphoru_rnc

path = '/kaggle/input/nlp-hw1/RNCgoldInUD_Morpho.conll'
records = load_morphoru_rnc(path)

In [14]:
corpus = {'ручка': [], 'коса': [], 'писать': [], 'свежий': [], 'больной': []}
records_used = {'ручка': [], 'коса': [], 'писать': [], 'свежий': [], 'больной': []}
for i, record in enumerate(records):
  for token in record.tokens:
    for word in corpus.keys():
      if token.lemma == word and i not in records_used[word]:
        corpus[word].append(record)
        records_used[word].append(i)

In [15]:
corpus['коса'][0]

MorphoSent(
    tokens=[MorphoToken(
         text='Она',
         lemma='она',
         pos='PRON',
         feats={'Case': 'Nom',
          'Gender': 'Fem',
          'Number': 'Sing',
          'Person': '3'},
         feats2={}
     ),
     MorphoToken(
         text='была',
         lemma='быть',
         pos='VERB',
         feats={'Gender': 'Fem',
          'Mood': 'Ind',
          'Number': 'Sing',
          'Tense': 'Past',
          'VerbForm': 'Fin',
          'Voice': 'Act'},
         feats2={'Subcat': 'Intr',
          'Aspect': 'Imp'}
     ),
     MorphoToken(
         text='ещё',
         lemma='еще',
         pos='ADV',
         feats={'Degree': 'Pos'},
         feats2={}
     ),
     MorphoToken(
         text='молодая',
         lemma='молодой',
         pos='ADJ',
         feats={'Case': 'Nom',
          'Gender': 'Fem',
          'Number': 'Sing',
          'Variant': 'Full'},
         feats2={}
     ),
     MorphoToken(
         text=',',
         lemma=',',
      

In [16]:
print('Количество предложений со словом "ручка":', len(corpus['ручка']))
print('Количество предложений со словом "коса":', len(corpus['коса']))
print('Количество предложений со словом "писать":', len(corpus['писать']))
print('Количество предложений со словом "свежий":', len(corpus['свежий']))
print('Количество предложений со словом "больной":', len(corpus['больной']))

Количество предложений со словом "ручка": 60
Количество предложений со словом "коса": 11
Количество предложений со словом "писать": 294
Количество предложений со словом "свежий": 84
Количество предложений со словом "больной": 146


# 3. Разбивка корпуса по значениям для каждой лексемы

## 3.1 Adagram

In [17]:
!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > all.a010.p10.d300.w5.m100.nonorm.slim.joblib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394M  100 1394M    0     0  50.3M      0  0:00:27  0:00:27 --:--:-- 49.6M


In [18]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [19]:
corpus_df = pd.DataFrame(columns=["lexeme", "sent", "sent_lemmatized", "ada_meaning_id"])

Т. к. корпус уже размечен морфологически, я взяла эту разметку и составила два текста: нормальный (как он бы выглядел просто на письме) и лемматизированный (просто леммы через пробел).

In [20]:
for lexeme in corpus.keys():
  for sent in corpus[lexeme]:
    sent_plain = sent.tokens[0].text
    for i in range(1, len(sent.tokens)):
      if sent.tokens[i].pos == 'PUNCT' and sent.tokens[i].text is not None:
        sent_plain += sent.tokens[i].text
      elif sent.tokens[i].text is not None:
        sent_plain += ' ' + sent.tokens[i].text

    sent_lemmatized = ''
    for token in sent.tokens:
      if token.pos != 'PUNCT':
        sent_lemmatized += ' ' + token.lemma
    sent_lemmatized = sent_lemmatized.strip()

    id = np.argmax(vm.disambiguate(lexeme, sent_lemmatized))
    corpus_df = pd.concat([pd.DataFrame([[lexeme, sent_plain, sent_lemmatized, id]], columns=corpus_df.columns), corpus_df], ignore_index=True)

In [21]:
corpus_df

,lexeme,sent,sent_lemmatized,ada_meaning_id
0,больной,Выявлять неблагополучные квартиры/ выявлять лю...,выявлять неблагополучный квартира выявлять чел...,1
1,больной,В этом главный вопрос не для нас с тобой// для...,в это главный вопрос не для мы с ты для пенсио...,1
2,больной,Но не значит/ что если человек болен одним/ то...,но не значить что если человек больной один то...,1
3,больной,Вот приходит больной/ а ему/ а/ нет/ помойся и...,вот приходить больной а он а нет помыться и пр...,1
4,больной,Мы все больны!,мы весь больной,3
...,...,...,...,...
590,ручка,Он подымает тётушкину мотыгу и одним ударом об...,он поднимать тетушкин мотыга и один удар обух ...,3
591,ручка,"-- Что же ты, обещал мне новую ручку приделать...",что же ты обещать я новый ручка приделать к мо...,3
592,ручка,"Я ухватился одной рукой за выбоину в стене, др...",я ухватиться один рука за выбоина в стена друг...,3
593,ручка,"Расшатав, гражданин вытащил чавкнувший прощаль...",расшатать гражданин вытащить чавкнуть прощальн...,3


## 3.2. Elmo

In [22]:
model = ElmoModel()
model.load("/kaggle/input/nlp-hw1/196", limit=50, max_batch_size=16)

'The model is now loaded.'

In [23]:
corpus_df['elmo_meaning_id'] = None

In [24]:
def get_elmo_vectors(word, contexts, model):
    all_vectors = model.get_elmo_vectors(contexts)
    word_vecs = []
    errors = []
    for i in range(len(contexts)):
        try:
            word_vecs.append(all_vectors[i][contexts[i].index(word)])
        except ValueError:
            errors.append(i)
            #список контекстов, для которых не нашелся эмбеддинг
    return word_vecs, errors

In [25]:
lexemes = []
for lex in corpus_df['lexeme']:
  if lex not in lexemes:
    lexemes.append(lex)

In [26]:
lexemes

['больной', 'свежий', 'писать', 'коса', 'ручка']

In [27]:
from sklearn.cluster import KMeans

В `kmeans` я запишу модели кластеризации для каждого слова, чтобы потом их применить к толкованиям. Это нужно, чтобы толкования "скластеризовались" рядом с правильными кластерами значений и получили правильный номер значения. Иначе нам будет не с чем кластеризовать толкования и мы не сможем понять правильность разметки значений Elmo.

In [28]:
kmeans = {}

Я беру `n_clusters = len(vm.word_sense_probs(lex))`, чтобы было проще сравнивать модели. То есть у Elmo получится столько же значений, сколько у Adagram для каждого слова.

In [29]:
for lex in lexemes:
  sents = list(corpus_df[corpus_df['lexeme'] == lex]['sent_lemmatized'])
  vecs, err = get_elmo_vectors(lex, sents, model)

  id0 = corpus_df.loc[corpus_df["lexeme"] == lex].index[0]
  id1 = corpus_df.loc[corpus_df["lexeme"] == lex].index[-1]
  n = len(vm.word_sense_probs(lex))
  kmeans_vec = KMeans(n_clusters=n).fit(vecs)
  kmeans[lex] = kmeans_vec
  labels = np.array(kmeans_vec.labels_)
  for i in err:
    labels = np.insert(labels, i, -1)
  corpus_df.loc[id0:id1, 'elmo_meaning_id'] = labels

In [30]:
corpus_df

,lexeme,sent,sent_lemmatized,ada_meaning_id,elmo_meaning_id
0,больной,Выявлять неблагополучные квартиры/ выявлять лю...,выявлять неблагополучный квартира выявлять чел...,1,3
1,больной,В этом главный вопрос не для нас с тобой// для...,в это главный вопрос не для мы с ты для пенсио...,1,0
2,больной,Но не значит/ что если человек болен одним/ то...,но не значить что если человек больной один то...,1,3
3,больной,Вот приходит больной/ а ему/ а/ нет/ помойся и...,вот приходить больной а он а нет помыться и пр...,1,3
4,больной,Мы все больны!,мы весь больной,3,2
...,...,...,...,...,...
590,ручка,Он подымает тётушкину мотыгу и одним ударом об...,он поднимать тетушкин мотыга и один удар обух ...,3,1
591,ручка,"-- Что же ты, обещал мне новую ручку приделать...",что же ты обещать я новый ручка приделать к мо...,3,2
592,ручка,"Я ухватился одной рукой за выбоину в стене, др...",я ухватиться один рука за выбоина в стена друг...,3,0
593,ручка,"Расшатав, гражданин вытащил чавкнувший прощаль...",расшатать гражданин вытащить чавкнуть прощальн...,3,0


In [31]:
corpus_df.to_csv('corpus.csv', encoding='utf-8')

# 4. Значение толкований

## 4.1. Adagram

Добавим слово в начало каждого толкования, потому что оно должно быть в контексте, значение которого мы хотим получить (для Elmo точно нужно, для Adagram мне казалось идеалогически тоже?)

Я еще добавляю два фиктивных значения для 'больного', потому что так получилось, что словарных значений 3, а у Adagram их 5, из-за чего в Elmo их тоже 5, и он не может кластеризовать 3 контекста в 5 кластеров.

In [32]:
def_df = pd.DataFrame(columns=["lexeme", "meaning_id_MAS", "meaning"])
for word in definitions.keys():
    for i, defin in enumerate(definitions[word]):
        def_df = pd.concat([pd.DataFrame([[word, i, word + ' — ' + defin]], columns=def_df.columns), def_df], ignore_index=True)
    
    if word == 'больной':
        def_df = pd.concat([pd.DataFrame([[word, -1, 'больной']], columns=def_df.columns), def_df], ignore_index=True)
        def_df = pd.concat([pd.DataFrame([[word, -1, 'больной']], columns=def_df.columns), def_df], ignore_index=True)

In [33]:
def_df

,lexeme,meaning_id_MAS,meaning
0,больной,-1,больной
1,больной,-1,больной
2,больной,2,"больной — Неестественный, ненормальный."
3,больной,1,больной — Свидетельствующий о болезни; болезне...
4,больной,0,больной — Страдающий какой-либо болезнью; прот...
5,свежий,8,"свежий — Морское: Сильный (о ветре, волне). С ..."
6,свежий,7,"свежий — Довольно холодный, прохладный."
7,свежий,6,"свежий — Недавно или только что появившийся, в..."
8,свежий,5,"свежий — Полный здоровья, сил (о человеке); цв..."
9,свежий,4,"свежий — Не потерявший своей силы, яркости, бл..."


In [34]:
def_df['ada_meaning_id'] = None
def_df['elmo_meaning_id'] = None

In [35]:
morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

mystem = Mystem()

def lemmatized_context(texts):
    lemmatized = []
    for s in texts:
        lemm = [w.lower() for w in mystem.lemmatize(" ".join(tokenize(s)))]
        lemm = ''.join(lemm).strip()
        lemmatized.append(lemm)
    return lemmatized

In [36]:
meaning_lemm = lemmatized_context(list(def_df.meaning))
meaning_lemm

['больной',
 'больной',
 'больной неестественный ненормальный',
 'больной свидетельствовать о болезнь болезненный',
 'больной страдающий какой либо болезнь противоположный здоровый',
 'свежий морской сильный о ветер волна с сильный холодный ветер о погода',
 'свежий довольно холодный прохладный',
 'свежий недавно или только что появляться возникать впервые или только что появляться где либо самый новый последний еще не известный кто либо впервые высказывать применять оригинальный',
 'свежий полный здоровье сила о человек цвести свидетельствовать о здоровье',
 'свежий не потерять свой сила яркость блеск или естественный здоровый окраска звучный звонкий чистый о голос смех и т п отчетливо и ясно сохраняться в память не забывать',
 'свежий восстанавливать обновлять отдых сон',
 'свежий ничто не загрязнять чистый часто обновляться',
 'свежий не бывший в употребление еще не применяться не использовать',
 'свежий недавно или только что получать добывать приготавливать и т п и не потерять сво

In [37]:
ada_ids = []
for meaning in meaning_lemm:
    word = meaning.split()[0]
    if meaning == 'больной':
        ada = -1
    else:
        ada = np.argmax(vm.disambiguate(word, meaning))
    ada_ids.append(ada)

In [38]:
def_df['ada_meaning_id'] = ada_ids
def_df

,lexeme,meaning_id_MAS,meaning,ada_meaning_id,elmo_meaning_id
0,больной,-1,больной,-1,None
1,больной,-1,больной,-1,None
2,больной,2,"больной — Неестественный, ненормальный.",3,None
3,больной,1,больной — Свидетельствующий о болезни; болезне...,1,None
4,больной,0,больной — Страдающий какой-либо болезнью; прот...,1,None
5,свежий,8,"свежий — Морское: Сильный (о ветре, волне). С ...",4,None
6,свежий,7,"свежий — Довольно холодный, прохладный.",4,None
7,свежий,6,"свежий — Недавно или только что появившийся, в...",4,None
8,свежий,5,"свежий — Полный здоровья, сил (о человеке); цв...",4,None
9,свежий,4,"свежий — Не потерявший своей силы, яркости, бл...",4,None


## 4.2. Elmo

In [39]:
lemm_dict = {'больной': [], 'свежий': [], 'писать': [], 'коса': [], 'ручка': []}
for meaning in meaning_lemm:
    word = meaning.split()[0]
    lemm_dict[word].append(meaning)

In [40]:
lemm_dict

{'больной': ['больной',
  'больной',
  'больной неестественный ненормальный',
  'больной свидетельствовать о болезнь болезненный',
  'больной страдающий какой либо болезнь противоположный здоровый'],
 'свежий': ['свежий морской сильный о ветер волна с сильный холодный ветер о погода',
  'свежий довольно холодный прохладный',
  'свежий недавно или только что появляться возникать впервые или только что появляться где либо самый новый последний еще не известный кто либо впервые высказывать применять оригинальный',
  'свежий полный здоровье сила о человек цвести свидетельствовать о здоровье',
  'свежий не потерять свой сила яркость блеск или естественный здоровый окраска звучный звонкий чистый о голос смех и т п отчетливо и ясно сохраняться в память не забывать',
  'свежий восстанавливать обновлять отдых сон',
  'свежий ничто не загрязнять чистый часто обновляться',
  'свежий не бывший в употребление еще не применяться не использовать',
  'свежий недавно или только что получать добывать пр

In [41]:
for lex in lemm_dict.keys():
  vecs, err = get_elmo_vectors(lex, lemm_dict[lex], model)

  id0 = def_df.loc[def_df["lexeme"] == lex].index[0]
  id1 = def_df.loc[def_df["lexeme"] == lex].index[-1]
  kmeans_new = kmeans[lex].fit(vecs)
  print(kmeans_new.labels_)
  labels = np.array(kmeans_new.labels_)
  for i in err:
    labels = np.insert(labels, i, -1)
  def_df.loc[id0:id1, 'elmo_meaning_id'] = labels

[1 1 0 2 3]
[2 1 2 2 0 4 3 3 0]
[0 3 2 0 4 1]
[0 2 1 3]
[0 2 3 1]


In [42]:
def_df

,lexeme,meaning_id_MAS,meaning,ada_meaning_id,elmo_meaning_id
0,больной,-1,больной,-1,1
1,больной,-1,больной,-1,1
2,больной,2,"больной — Неестественный, ненормальный.",3,0
3,больной,1,больной — Свидетельствующий о болезни; болезне...,1,2
4,больной,0,больной — Страдающий какой-либо болезнью; прот...,1,3
5,свежий,8,"свежий — Морское: Сильный (о ветре, волне). С ...",4,2
6,свежий,7,"свежий — Довольно холодный, прохладный.",4,1
7,свежий,6,"свежий — Недавно или только что появившийся, в...",4,2
8,свежий,5,"свежий — Полный здоровья, сил (о человеке); цв...",4,2
9,свежий,4,"свежий — Не потерявший своей силы, яркости, бл...",4,0


# 5. Совпадение значения слова в предложении со значением толкования

Будем считать, что если модель определила несколько словарных значений как одно, то мы слушаем ее, а не словарь.

## 5.1. Adagram

In [43]:
ada_true_df = pd.DataFrame(columns=["lexeme", "context", "ada_meaning_id", "ada_TRUE"])

In [44]:
for lex in lexemes:
    df = corpus_df[corpus_df['lexeme'] == lex]
    meanings = set(df.ada_meaning_id)
    for mean in meanings:
        df_ = list(df[df['ada_meaning_id'] == mean].sent)
        cnt = 0
        for cont in df_:
            ada_true_df = pd.concat([pd.DataFrame([[lex, cont, mean, None]], columns=ada_true_df.columns), ada_true_df], ignore_index=True)
            cnt += 1
            if cnt == 5:
                break

In [45]:
ada_true_df

,lexeme,context,ada_meaning_id,ada_TRUE
0,ручка,Очень часто в кулуарах было такое/ вы нам дайт...,3,None
1,ручка,Сейчас бизнесу дали эту ручку двери/ и насколь...,3,None
2,ручка,А до этого просто ручка.,3,None
3,ручка,И за ручку/ за ручку отвести в другое место.,3,None
4,ручка,Я работаю токарем на заводе/ а чтоб крутить эт...,3,None
5,ручка,Никогда не думали о том/ что возьмёте ручку/ с...,2,None
6,коса,"Это пришла Мила Улыбкина, худенькая девочка с ...",3,None
7,коса,"Она неопределённо фыркнула, перебросила через ...",3,None
8,коса,"Но мне, в принципе, неважно, где выступать-- м...",3,None
9,коса,Здесь нашла коса на камень: прорыв 34-й армии ...,3,None


In [46]:
for i, row in ada_true_df.iterrows():
    print(ada_true_df.at[i, 'context'])
    print('НОМЕР ЗНАЧЕНИЯ:', ada_true_df.at[i, 'ada_meaning_id'])
    print('ЛЕКСЕМА:', ada_true_df.at[i, 'lexeme'], '\n')
    true = int(input())
    ada_true_df.at[i, 'ada_TRUE'] = true

Очень часто в кулуарах было такое/ вы нам дайте схватиться за ручку двери/ а мы уж дверь сами вышибем.
НОМЕР ЗНАЧЕНИЯ: 3
ЛЕКСЕМА: ручка 



 1


Сейчас бизнесу дали эту ручку двери/ и насколько хватит потенции/ насколько хватит согласованности команды/ насколько хватит подготовленности команды к использованию этого хорошего настроения и конструктивной позиции государства в реальных целях/ это очень большой вопрос.
НОМЕР ЗНАЧЕНИЯ: 3
ЛЕКСЕМА: ручка 



 1


А до этого просто ручка.
НОМЕР ЗНАЧЕНИЯ: 3
ЛЕКСЕМА: ручка 



...

 1


В этом главный вопрос не для нас с тобой// для пенсионеров/ для больных/ для слабых людей.
НОМЕР ЗНАЧЕНИЯ: 1
ЛЕКСЕМА: больной 



 1


Выявлять неблагополучные квартиры/ выявлять людей больных/ немощных,/ брошенных детей/ которых алкоголики бросают и уходят гулять/ а дети одни в квартире.
НОМЕР ЗНАЧЕНИЯ: 1
ЛЕКСЕМА: больной 



 1


## 5.2. Elmo

In [47]:
elmo_true_df = pd.DataFrame(columns=["lexeme", "context", "elmo_meaning_id", "elmo_TRUE"])

In [48]:
for lex in lexemes:
    df = corpus_df[corpus_df['lexeme'] == lex]
    meanings = set(df.elmo_meaning_id)
    for mean in meanings:
        df_ = list(df[df['elmo_meaning_id'] == mean].sent)[:2]
        cnt = 0
        for cont in df_:
            elmo_true_df = pd.concat([pd.DataFrame([[lex, cont, mean, None]], columns=elmo_true_df.columns), elmo_true_df], ignore_index=True)
            cnt += 1
            if cnt == 5:
                break

In [50]:
for i, row in elmo_true_df.iterrows():
    print(elmo_true_df.at[i, 'context'])
    print('НОМЕР ЗНАЧЕНИЯ:', elmo_true_df.at[i, 'elmo_meaning_id'])
    print('ЛЕКСЕМА:', elmo_true_df.at[i, 'lexeme'], '\n')
    true = int(input())
    elmo_true_df.at[i, 'elmo_TRUE'] = true

Ручка, ножка, огуречик...
НОМЕР ЗНАЧЕНИЯ: 3
ЛЕКСЕМА: ручка 



 1


Ручка у неё некрепкая.
НОМЕР ЗНАЧЕНИЯ: 3
ЛЕКСЕМА: ручка 



 1


Я работаю токарем на заводе/ а чтоб крутить эти ручки надо что-то кушать.
НОМЕР ЗНАЧЕНИЯ: 2
ЛЕКСЕМА: ручка 



...

 0


А Северная Корея гораздо больнее/ чем наша страна.
НОМЕР ЗНАЧЕНИЯ: 0
ЛЕКСЕМА: больной 



 1


В этом главный вопрос не для нас с тобой// для пенсионеров/ для больных/ для слабых людей.
НОМЕР ЗНАЧЕНИЯ: 0
ЛЕКСЕМА: больной 



 0


In [52]:
elmo_true_df

,lexeme,context,elmo_meaning_id,elmo_TRUE
0,ручка,"Ручка, ножка, огуречик...",3,1
1,ручка,Ручка у неё некрепкая.,3,1
2,ручка,Я работаю токарем на заводе/ а чтоб крутить эт...,2,0
3,ручка,Никогда не думали о том/ что возьмёте ручку/ с...,2,0
4,ручка,Но Марина уже поднимает правую ручку!,1,1
5,ручка,А до этого просто ручка.,1,1
6,ручка,Очень часто в кулуарах было такое/ вы нам дайт...,0,0
7,ручка,И за ручку/ за ручку отвести в другое место.,0,0
8,коса,"Она неопределённо фыркнула, перебросила через ...",3,1
9,коса,Здесь нашла коса на камень: прорыв 34-й армии ...,3,0


In [51]:
ada_true_df.to_csv('ada_true.csv', encoding='utf-8')
elmo_true_df.to_csv('elmo_true.csv', encoding='utf-8')

# 6. Accuracy

In [53]:
ada_list = list(ada_true_df.ada_TRUE)
ada_accuracy = sum(ada_list) / len(ada_list)
ada_accuracy

0.6666666666666666

In [55]:
elmo_list = list(elmo_true_df.elmo_TRUE)
elmo_accuracy = sum(elmo_list) / len(elmo_list)
elmo_accuracy

0.38636363636363635

# 7. Оценка результатов

У Adagram сравнительно неплохой показатель accuracy, но он достигается несколько читерским способом. Очень многие толкования оказались объединены в одно значение (так, все 6 значений 'писать' оказались одним значением, аналогично с 9 значениями 'свежего' и 4 значениями 'ручки'). Максимум Adagram выделил два значения у толкований и то странным образом - объединил все и очень разные значения 'косы' в одно и только отмель отметил как отдельное. Неплохим мне показалось только выделение значений слова 'больной'.
Итак, благодаря тому, что почти все слова оказались однозначными, пришлось отмечать контексты как правильно истолкованные, хотя, действительности это не соответствует.

У Elmo accuracy довольно плохая.

Причина, почему у Elmo плохая accuracy и почему Adagram так плохо распознал значения толкований кроется в специфике толкований. Кажется, что это не очень прототипический контекст. Во-первых, в нем почти никогда не встречается само слово (что как раз нужно для этих моделей). Во-вторых, в толкованиях также редко встречаются слова, которые обычно встречаются часто рядом с исследуемым словом - контекст. Возможно влияет также то, что встречаются синонимичные слова для объяснения смысла слова.
Решением этой проблемы могло бы стать включение примеров из словаря для каждого значения, как в МАС: -- *Да позвольте, как же мне писать расписку? прежде нужно видеть деньги*. Гоголь, Мертвые души.
Если добавить эти контексты к толкованиям (или вообще заменить толкования ими), то, возможно, наши модели смогут работать лучше. Adagram будет лучше считывать значение "толкования" благодаря прототипичным контекстам для него, а Elmo -- лучше кластеризовать "толкования" с предложениями из корпуса, благодаря чему тоже лучше выделять значения.

Если сравнивать Adagram и Elmo по тому, как они справлялись с отдельными лексемами, то мне показалось, что Elmo почти всегда была лучше (но прямо количественно как-то я не сравнивала, это, скорее, импрессионистическое представление).